In [1]:
# Install PyTorch Geometric on Google Colab (run this cell once)
!pip install torch_geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.7 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

# Choose device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0].to(device)  # move data to device

# Define a simple 2-layer GCN
class SimpleGCN(torch.nn.Module):
    def init(self, in_channels, hidden_channels, out_channels):
        super().init()
        self.conv1 = GCNConv(in_channels, hidden_channels)   # 1st layer
        self.conv2 = GCNConv(hidden_channels, out_channels)  # 2nd layer

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        # First GCN layer + ReLU
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        # Second GCN layer
        x = self.conv2(x, edge_index)
        # Log softmax for NLL loss
        return F.log_softmax(x, dim=1)

# Create model and optimizer
model = SimpleGCN(
    in_channels=dataset.num_node_features,
    hidden_channels=16,
    out_channels=dataset.num_classes
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Training loop
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

# Test function
def test():
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)

    accs = {}
    for split, mask in [("train", data.train_mask),
                        ("val", data.val_mask),
                        ("test", data.test_mask)]:
        correct = (pred[mask] == data.y[mask]).sum().item()
        total = int(mask.sum())
        accs[split] = correct / total
    return accs

# Run training
for epoch in range(1, 201):  # 200 epochs
    loss = train()
    if epoch % 20 == 0:
        accs = test()
        print(f"Epoch: {epoch:03d} | Loss: {loss:.4f} | "
              f"Train Acc: {accs['train']:.2f} | Val Acc: {accs['val']:.2f} | Test Acc: {accs['test']:.2f}")

# Final test accuracy
accs = test()
print(f"\nFinal GCN Test Accuracy: {accs['test']:.2f}")

Using device: cpu


Processing...
Done!


TypeError: SimpleGCN.__init__() got an unexpected keyword argument 'in_channels'

In [3]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

# Choose device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0].to(device)  # move data to device


# ---------------------------
#   FIXED MODEL CLASS
# ---------------------------
class SimpleGCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


# Create model and optimizer
model = SimpleGCN(
    in_channels=dataset.num_node_features,
    hidden_channels=16,
    out_channels=dataset.num_classes
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


# Training loop
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()


# Test function
def test():
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)

    accs = {}
    for split, mask in [("train", data.train_mask),
                        ("val", data.val_mask),
                        ("test", data.test_mask)]:
        correct = (pred[mask] == data.y[mask]).sum().item()
        total = int(mask.sum())
        accs[split] = correct / total
    return accs


# Run training
for epoch in range(1, 201):
    loss = train()
    if epoch % 20 == 0:
        accs = test()
        print(f"Epoch: {epoch:03d} | Loss: {loss:.4f} | "
              f"Train Acc: {accs['train']:.2f} | Val Acc: {accs['val']:.2f} | Test Acc: {accs['test']:.2f}")

# Final accuracy
accs = test()
print(f"\nFinal GCN Test Accuracy: {accs['test']:.2f}")


Using device: cpu
Epoch: 020 | Loss: 0.0990 | Train Acc: 1.00 | Val Acc: 0.79 | Test Acc: 0.80
Epoch: 040 | Loss: 0.0136 | Train Acc: 1.00 | Val Acc: 0.79 | Test Acc: 0.80
Epoch: 060 | Loss: 0.0134 | Train Acc: 1.00 | Val Acc: 0.79 | Test Acc: 0.80
Epoch: 080 | Loss: 0.0161 | Train Acc: 1.00 | Val Acc: 0.79 | Test Acc: 0.81
Epoch: 100 | Loss: 0.0151 | Train Acc: 1.00 | Val Acc: 0.78 | Test Acc: 0.81
Epoch: 120 | Loss: 0.0135 | Train Acc: 1.00 | Val Acc: 0.78 | Test Acc: 0.81
Epoch: 140 | Loss: 0.0123 | Train Acc: 1.00 | Val Acc: 0.78 | Test Acc: 0.81
Epoch: 160 | Loss: 0.0114 | Train Acc: 1.00 | Val Acc: 0.78 | Test Acc: 0.81
Epoch: 180 | Loss: 0.0106 | Train Acc: 1.00 | Val Acc: 0.78 | Test Acc: 0.81
Epoch: 200 | Loss: 0.0100 | Train Acc: 1.00 | Val Acc: 0.77 | Test Acc: 0.81

Final GCN Test Accuracy: 0.81


In [4]:
!pip install torch_geometric

import networkx as nx
import torch
from torch_geometric.data import Data

# Create a small social graph
G = nx.Graph()
edges = [(0,1), (1,2), (2,0),    # normal users community
         (3,4), (4,5), (5, 3)]    # suspicious cluster
G.add_edges_from(edges)

# One-hot node features (6 nodes, 6-dimensional)
num_nodes = 6
features = torch.eye(num_nodes, dtype=torch.float)

# Node labels: first 3 normal (0), last 3 bots (1)
labels = torch.tensor([0,0,0, 1,1,1], dtype=torch.long)

# Convert to PyG Data
edge_index = torch.tensor(list(G.edges())).t().contiguous()
# Make edges bidirectional for undirected graph
edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)
data = Data(x=features, edge_index=edge_index, y=labels)

# Define train/test masks (e.g., train on 4 nodes, test on 2 nodes)
data.train_mask = torch.tensor([1,1,1,1,0,0], dtype=torch.bool)
data.test_mask  = torch.tensor([0,0,0,0,1,1], dtype=torch.bool)

In [52]:
import torch
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid

# Load a sample dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

class SimpleGCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Use data.num_node_features after data is defined
        self.conv1 = GCNConv(data.num_node_features, 4)  # 1st layer: in_dim->4
        self.conv2 = GCNConv(4, dataset.num_classes)     # 2nd layer: 4->num_classes

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))  # aggregate + activate
        x = self.conv2(x, edge_index)          # second layer
        return F.log_softmax(x, dim=1)

model = SimpleGCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop (simplified)
for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

# Evaluate accuracy on test nodes
model.eval()
pred = model(data).argmax(dim=1)
acc = (pred[data.test_mask] == data.y[data.test_mask]).float().mean()
print(f'GCN Test Accuracy: {acc:.2f}')


GCN Test Accuracy: 0.65
